In [1]:
from azureml.core import Environment

env = Environment(name="azure_pytorch_env")
env.python.conda_dependencies.add_pip_package("torch==2.1.0")
env.python.conda_dependencies.add_pip_package("numpy")
env.python.conda_dependencies.add_pip_package("scikit-learn")

In [2]:
from azureml.core import Workspace, Model
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Connect to Azure ML Workspace
ws = Workspace.from_config()

# Load registered model and scaler
model = Model(ws, "pytorch_nn_gru")
scaler = Model(ws, "adbl_scaler")


# Define inference configuration
inference_config = InferenceConfig(
    entry_script="score.py",
    environment=env
)

# Define deployment configuration (ACI)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# Deploy both the model and scaler
service = Model.deploy(
    workspace=ws,
    name="pytorch-n-gru-scalar",
    models=[model, scaler],  # <-- Include both
    inference_config=inference_config,
    deployment_config=deployment_config
)

service.wait_for_deployment(show_output=True)

# Print Scoring URI
print(f"Deployment successful! Scoring URI: {service.scoring_uri}")


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.
/tmp/ipykernel_101937/890583880.py:23: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-04-06 21:57:31+05:45 Creating Container Registry if not exists.
2025-04-06 21:57:34+05:45 Use the existing image.
2025-04-06 21:57:34+05:45 Generating deployment configuration.
2025-04-06 21:57:38+05:45 Submitting deployment to compute.
2025-04-06 21:57:43+05:45 Checking the status of deployment pytorch-n-gru-scalar..
2025-04-06 22:00:17+05:45 Checking the status of inference endpoint pytorch-n-gru-scalar.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Deployment successful! Scoring URI: http://55648af8-0be3-4b42-bfa7-89419dfbd74d.eastus2.azurecontainer.io/score


In [3]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_51c067e992bfb00fa57ba7a838831dea/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_51c067e992bfb00fa57ba7a838831dea/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_51c067e992bfb00fa57ba7a838831dea/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2025-04-06T16:15:04,029917213+00:00 - rsyslog/run 
2025-04-06T16:15:04,034857794+00:00 - gunicorn/run 
bash: /azureml-envs/azureml_51c067e992bfb00fa57ba7a838831dea/lib/libtinfo.so.6: no version information available (required by bash)
2025-04-06T16:15:04,038302235+00:00 | gunicorn/run | 
2025-04-06T16:15:04,041125968+00:00 | gunicorn/run | ###############################################
2025-04-06T16:15:04,043706329+00:00 | gunicorn/run | AzureML Container Runtime Information
2025-04-06T16:15:04,045128169+00:00 - nginx/run 
2025-04-06T16:15:04,046297477+00:00 | gunico

In [4]:
import requests
import json

# Select a sample (first one for testing)

sample_input = [[0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1]]

# Define the input JSON payload
payload = json.dumps({"data": sample_input})

# Get the deployment endpoint
scoring_uri = service.scoring_uri
headers = {"Content-Type": "application/json"}

# Send request to the deployed model
response = requests.post(scoring_uri, data=payload, headers=headers)

# Print response
print("Response:", response.json())

Response: [385.9725646972656, 385.8465270996094]
